# 16 MONTE CARLO SIMULATION
Stanislaw Ulam and Nicholas Metropolis coined the term <b>Monte Carlo simulation</b> in 1949 in homage to the games of chance played in the casino in the Principality of Monaco.

Stanislaw Ulam：

```
I wondered whether a more practical method than “abstract thinking” might not be to lay it out say 
one hundred times and simply observe and count the number of successful plays

This was already possible to envisage with the beginning of the new era of fast computers

```

## 16.1 Pascal’s Problem

Pascal’s interest in the field that came to be known as probability theory began when a friend asked him <b>whether or not it would be profitable to bet that within twenty-four rolls of a pair of dice he would roll a double six</b>

In the long run it <b>would not be profitable to bet</b> on rolling a double six within twenty-four rolls



In [ ]:
import random

def rollDie():
    return random.choice([1,2,3,4,5,6])

def checkPascal(numTrials):
    """Assumes numTrials an int > 0
       Prints an estimate of the probability of winning"""
    numWins = 0.0
    for i in range(numTrials):
        for j in range(24):
            d1 = rollDie()
            d2 = rollDie()
            if d1 == 6 and d2 == 6:
                numWins += 1
                break
    print('Probability of winning =', numWins/numTrials)

checkPascal(1000)

In [ ]:
1 - (35.0/36.0)**24

## 16.2 Pass or Don’t Pass?

In the game craps, the shooter (the person who rolls the dice) chooses between making a “pass line” or a “don’t pass line” bet.

<b>Pass Line</b>

1)Shooter wins if the first roll (called coming out) is a “natural” (7 or 11) 

2) loses if it is “craps” (2, 3, or 12). 

3) If some other number is rolled, that number becomes <b>the “point”</b> and the shooter <b>keeps rolling</b>.
     
        If the shooter rolls `the point` before rolling a 7, the shooter wins. Otherwise  the shooter loses.

<b>Don’t Pass Line</b>: 

1) Shooter loses if the first roll is 7 or 11, 
  
2) wins if it is 2 or 3,

3) ties (a “push” in gambling jargon) if it is 12. 

4) If some other number is rolled, that number becomes the point and shooter keeps rolling.

       If the shooter rolls a 7 before rolling the point, the shooter wins. Otherwise the shooter loses.

#### Is one of these a better bet than the other? Is either a good bet?

It seems easier to write a program that simulates a craps game, and see what happens.

In [ ]:
class CrapsGame(object):
    
    def __init__(self):
        #pass line
        self.passWins, self.passLosses = (0,0)
        # don't pass line 
        self.dpWins, self.dpLosses, self.dpPushes = (0,0,0)

    def playHand(self):
        throw = rollDie() + rollDie()
        if throw == 7 or throw == 11:
            self.passWins += 1
            self.dpLosses += 1
        elif throw == 2 or throw == 3 or throw == 12:
            self.passLosses += 1
            if throw == 12:  # ties
                self.dpPushes += 1
            else:
                self.dpWins += 1
        else:
            # a point is established, the shooter keeps rolling
            point = throw
            while True:
                throw = rollDie() + rollDie()
                if throw == point:
                    self.passWins += 1
                    self.dpLosses += 1
                    break
                elif throw == 7:
                    self.passLosses += 1
                    self.dpWins += 1
                    break

    def passResults(self):
        return (self.passWins, self.passLosses)

    def dpResults(self):
        return (self.dpWins, self.dpLosses, self.dpPushes)

Figure 14.3 contains a function that uses class CrapsGame. Its structure is
typical of many simulation programs:

1. It runs multiple games (think of each game as analogous to a trial in our earlier simulations) and accumulates the results. Each game includes multiple hands, so there is a nested loop.

2. It then produces and stores statistics for each game.

3. Finally it produces and outputs summary statistics. In this case, it  prints the expected return on investment (ROI) or each kind of betting line and the standard deviation of that ROI

Rreturn on investment (ROI):

$ROI=\frac{gain from investment - cost of investment}{cost of investment}$

In [7]:
def variance(X):
    """Assumes that X is a list of numbers.
       Returns the standard deviation of X"""
    mean = sum(X)/len(X)
    tot = 0.0
    for x in X:
        tot += (x - mean)**2
    return tot/len(X)
    
def stdDev(X):
    """Assumes that X is a list of numbers.
       Returns the standard deviation of X"""
    return variance(X)**0.5

def crapsSim(handsPerGame, numGames):
    """Assumes handsPerGame and numGames are ints > 0
       Play numGames games of handsPerGame hands, and print results"""
    games = []

    #1 Play numGames games
    for t in range(numGames):
        c = CrapsGame()
        for i in range(handsPerGame):
            c.playHand()
        games.append(c)
        
    #2 Produce statistics for each game
    pROIPerGame, dpROIPerGame = [], []
    for g in games:
        wins, losses = g.passResults()
        pROIPerGame.append((wins - losses)/float(handsPerGame))
        
        wins, losses, pushes = g.dpResults()
        dpROIPerGame.append((wins - losses)/float(handsPerGame))
        
    #3 Produce and print summary statistics
    meanROI = str(round((100.0*sum(pROIPerGame)/numGames), 4)) + '%'
    sigma = str(round(100.0*stdDev(pROIPerGame), 4)) + '%'
    print('Pass:', 'Mean ROI =', meanROI, 'Std. Dev. =', sigma)
    
    meanROI = str(round((100.0*sum(dpROIPerGame)/numGames), 4)) + '%'
    sigma = str(round(100.0*stdDev(dpROIPerGame), 4)) + '%'
    print('Don\'t pass:','Mean ROI =', meanROI, 'Std Dev =', sigma)

Let’s run our craps simulation and see what happens

In [ ]:
crapsSim(20, 10)

increasing the number of hands per game,

In [ ]:
crapsSim(1000, 10)

we increased the number of games,

In [ ]:
crapsSim(20, 1000)

#### One of the nice things about simulations is that they make it easy to perform “what if” experiments.

For example, what if a player could sneak in a pair of cheater’s dice that favored 5 over 2


In [ ]:
import random


def rollDie():
    return random.choice([1,1,2,3,3,4,4,5,5,5,6,6])


In [ ]:
crapsSim(1000, 10)

## 16.3 Using Table Lookup to Improve Performance

It takes a long time to complete on most computers. That raises the question of whether
there is a simple way to speed up the simulation.

The complexity of `crapsSim` is `O(playHand)*handsPerGame*numGames`. The running time of playHand depends upon the number of times the loop in it is executed

For each possible point, one can easily calculate the probability of rolling that point before rolling a seven.

We have <b>pre-computed the probability</b> of making the point before rolling a 7 for each possible value of the point, and <b>stored those values in a dictionary</b>.

<b>Having this table allows us to replace the inner loop</b>, which contained an unbounded number of rolls,
with a test against one call to random.random. The asymptotic complexity of this version of playHand is <b>O(1)</b>.


The idea of replacing computation by <b>table lookup</b> has broad applicability and is frequently used when speed is an issue. Table lookup is an example of the general idea of <b>trading time for space</b>.

Using table lookup to improve performance

In [8]:
class FastCrapsGame(CrapsGame):
    
    def playHand(self):
        #An alternative, faster, implementation of playHand
        # pre-computed the probability 
        pointsDict = {4:1/3.0, 5:2/5.0, 6:5/11.0, 8:5/11.0, 9:2/5.0, 10:1/3.0}
       
        throw = rollDie() + rollDie()
        if throw == 7 or throw == 11:
            self.passWins += 1
            self.dpLosses += 1
        elif throw == 2 or throw == 3 or throw == 12:
            self.passLosses += 1
            if throw == 12:
                self.dpPushes += 1
            else:
                self.dpWins += 1
        else:
            # Having this table allows us to replace the inner loop
            if random.random() <= pointsDict[throw]: # point before 7
                self.passWins += 1
                self.dpLosses += 1
            else:                                    # 7 before point
                self.passLosses += 1
                self.dpWins += 1


NameError: name 'CrapsGame' is not defined

In [ ]:
def variance(X):
    """Assumes that X is a list of numbers.
       Returns the standard deviation of X"""
    mean = sum(X)/len(X)
    tot = 0.0
    for x in X:
        tot += (x - mean)**2
    return tot/len(X)
    
def stdDev(X):
    """Assumes that X is a list of numbers.
       Returns the standard deviation of X"""
    return variance(X)**0.5

def FastcrapsSim(handsPerGame, numGames):
    """Assumes handsPerGame and numGames are ints > 0
       Play numGames games of handsPerGame hands, and print results"""
    games = []

    #Play numGames games
    for t in range(numGames):
        c = FastCrapsGame()
        for i in range(handsPerGame):
            c.playHand()
        games.append(c)
        
    #Produce statistics for each game
    pROIPerGame, dpROIPerGame = [], []
    for g in games:
        wins, losses = g.passResults()
        pROIPerGame.append((wins - losses)/float(handsPerGame))
        wins, losses, pushes = g.dpResults()
        dpROIPerGame.append((wins - losses)/float(handsPerGame))
        
    #Produce and print summary statistics
    meanROI = str(round((100.0*sum(pROIPerGame)/numGames), 4)) + '%'
    sigma = str(round(100.0*stdDev(pROIPerGame), 4)) + '%'
    print('Pass:', 'Mean ROI =', meanROI, 'Std. Dev. =', sigma)
    meanROI = str(round((100.0*sum(dpROIPerGame)/numGames), 4)) + '%'
    sigma = str(round(100.0*stdDev(dpROIPerGame), 4)) + '%'
    print('Don\'t pass:','Mean ROI =', meanROI, 'Std Dev =', sigma)

In [ ]:
%time FastcrapsSim(5000, 10)

In [ ]:
%time crapsSim(5000, 10)

## 16.4 Finding $π$

It is easy to see how Monte Carlo simulation is useful for tackling problems in which nondeterminism plays a role. Interestingly, however, Monte Carlo simulation (and randomized algorithms in general) can be used to solve problems that are not inherently stochastic, i.e., for which there is no uncertainty about outcomes

###  Consider $π$

For thousands of years, people have known that there is a constant (called $π$ since the 18th century) such that the circumference of a circle is equal to $π*diameter$ and the area of the circle equal to $π * radius^2$. What they did not know was the value of this constant.

One of the earliest estimates, $4*(8/9)2 = 3.16$, can found in the **Egyptian Rhind Papyrus**, circa 1650 BC. More than a thousand years later, the **Old Testament** implied a different value for $π$ when giving the specifications of one of King Solomon’s

>And he made a molten sea, ten cubits from the one brim to the other: it was round all about, and his height was five cubits: and a line of >thirty cubits did compass it round about.

Solving for $π$, $10π = 30$, so $π = 3.$ Perhaps the **Bible** is simply wrong, or perhaps the molten sea wasn’t perfectly circular, or perhaps the circumference was measured from the outside of the wall and the diameter from the inside, or perhaps it’s just poetic
license. We leave it to the reader to decide.

**Archimedes of Syracuse** (287-212 BCE) derived upper and lower bounds on the value of $π$ by using a high-degree polygon to approximate a circular shape. Using a polygon with 96 sides, he concluded that $223/71 < π < 22/7$. Giving upper and lower bounds was a rather sophisticated approach for the time. Also, if we take his best estimate as the average of his two bounds we obtain $3.1418$, an error of about $0.0002$. **Not bad!**

Long before computers were invented, the French mathematicians **Buffon** (1707-1788) and **Laplace** (1749-1827) proposed using a stochastic simulation to estimate the value of $π$. Think about inscribing a circle in a square with sides of length $2$, so that the radius, $r$, of the circle is of length $1$.

![](./img/unit_circle.jpg)

By the definition of $π$, $area = π*r^2$. Since $r$ is $1$, $π = area$. But what’s the area of the circle? Buffon suggested that he could estimate the area of a circle by a dropping a large number of needles (which he argued would follow a random path as they fell) in the vicinity of the square. The ratio of the number of needles with tips lying within the square to the number of needles with tips lying within the circle could then be used to estimate the area of the circle.

If the locations of the needles are truly random, we know that

$\frac{needles in circle}{needles in square}=\frac{area of circle}{area of square}$

solving for the area of the circle,

${area of circle}={area of square}*\frac{needles in circle}{needles in square}$


Recall that the area of a $2$ by $2$ square is $4$, so,

${area of circle}=\frac{4*needles in circle}{needles in square}$

In general, to estimate the area of some region $R$

1. Pick an enclosing region, $E$, such that the area of $E$ is easy to calculate and $R$ lies completely within $E$.

2. Pick a set of random points that lie within $E.$

3. Let $F$ be the fraction of the points that fall within $R$.

4. Multiply the area of $E$ by $F.$

If you try **Buffon**’s experiment, you’ll soon realize that the places where the needles land are not truly random. Moreover, even if you could drop them randomly, it would take a very large number of needles to get an approximation of $π$ as good as even the
**Bible**’s. Fortunately, computers can randomly drop simulated needles at a ferocious rate.


The following code cell contains a program that estimates $π$ using the **Buffon-Laplace** method. For simplicity, it considers only those needles that fall in the upper right-hand quadrant of the square.

The function **throwNeedles** simulates dropping a needle by first using `random.random` to get a pair of positive Cartesian coordinates (x and y values) representing the position of the needle with respect to the center of the square. It then uses the` Pythagorean theorem` to compute the hypotenuse of the right triangle with base $x$ and height $y$. This is the distance of the tip of the needle from the origin (the center of the square). Since the radius of the circle is $1$, we know that the needle lies within the circle if and only if the distance from the origin is no greater than $1$. We use this fact to count the number of needles in the circle.

The function **getEst** uses **throwNeedles** to find an estimate of $π$ by first dropping `numNeedle`s needles, and then averaging the result over `numTrials` trials. It then returns the `mean` and `standard deviation` of the trials

The function **estPi** calls **getEst** with an ever-growing number of needles until the standard deviation returned by getEst is no larger than $precision/1.96$. Under the assumption that the errors are normally distributed, this implies that $95%$ of the values lie within recision of the mean.



In [10]:
import random

def variance(X):
    """Assumes that X is a list of numbers.
       Returns the standard deviation of X"""
    mean = sum(X)/len(X)
    tot = 0.0
    for x in X:
        tot += (x - mean)**2
    return tot/len(X)
    
def stdDev(X):
    """Assumes that X is a list of numbers.
       Returns the standard deviation of X"""
    return variance(X)**0.5

def throwNeedles(numNeedles):
    inCircle = 0
    for Needles in range(1, numNeedles + 1):
        x = random.random()
        y = random.random()
        if (x*x + y*y)**0.5 <= 1.0:
            inCircle += 1
    #Counting needles in one quadrant only, so multiply by 4
    return 4*(inCircle/float(numNeedles))

def getEst(numNeedles, numTrials):
    estimates = []
    for t in range(numTrials):
        piGuess = throwNeedles(numNeedles)
        estimates.append(piGuess)
    sDev = stdDev(estimates)
    curEst = sum(estimates)/len(estimates)
    return (curEst, sDev)

def estPi(precision, numTrials):
    numNeedles = 1000
    sDev = precision
    while sDev >= precision/1.96:
        curEst, sDev = getEst(numNeedles, numTrials)
        print('Est. = ' + str(round(curEst, 5)) +
          ', Std. dev. = ' + str(round(sDev, 5))
          + ', Needles = ' + str(numNeedles))
        numNeedles *= 2
    return curEst

When we ran `estPi(0.01, 100)` it printed
```text
Est. = 3.13792, Std. dev. = 0.05002, Needles = 1000
Est. = 3.14486, Std. dev. = 0.03405, Needles = 2000
Est. = 3.14114, Std. dev. = 0.02798, Needles = 4000
Est. = 3.14116, Std. dev. = 0.01992, Needles = 8000
Est. = 3.14327, Std. dev. = 0.01319, Needles = 16000
Est. = 3.14066, Std. dev. = 0.00881, Needles = 32000
Est. = 3.14248, Std. dev. = 0.0064, Needles = 64000
Est. = 3.141, Std. dev. = 0.00439, Needles = 12800
```

In [11]:
estPi(0.01, 100) 

Est. = 3.14124, Std. dev. = 0.05363, Needles = 1000
Est. = 3.14046, Std. dev. = 0.03327, Needles = 2000
Est. = 3.14226, Std. dev. = 0.02674, Needles = 4000
Est. = 3.13972, Std. dev. = 0.01798, Needles = 8000
Est. = 3.14272, Std. dev. = 0.01196, Needles = 16000
Est. = 3.14233, Std. dev. = 0.00919, Needles = 32000
Est. = 3.14001, Std. dev. = 0.00633, Needles = 64000
Est. = 3.14247, Std. dev. = 0.00398, Needles = 128000


3.1424709374999997

As one would expect, the standard deviations decreased monotonically as we increased the number of samples. In the beginning the estimates of the value of $π$ also improved steadily. Some were above the true value and some below, but each increase in numNeedles led to an improved estimate. With  $1000$ samples per trial, the simulation’s estimate was already better than those of the **Bible** and the **Rhind Papyrus.**

Curiously, the estimate got worse when the number of needles went from $8,000$ to $16,000$, since $3.14116$ is farther from the true value of $π$ than is $3.14327$. However, if we look at the ranges defined by one standard deviation around each of the means, both
ranges contain the true value of $π$, and the range associated with the larger sample size is smaller. Even though the estimate generated with $16,000$ samples happens to be farther from the actual value of $π$, we should have more confidence in its accuracy. This is an extremely important notion. It is not sufficient to produce a good answer. We have to have a valid reason to be confident that it is in fact a good answer. And when we drop a large enough number of needles, the small standard deviation gives us reason to
be confident that we have a correct answer. Right?

Not exactly. Having a small standard deviation is a necessary condition for having confidence in the validity of the result. It is not a sufficient condition. The notion of a statistically valid conclusion should never be confused with the notion of a correct conclusion.

Each statistical analysis starts with a set of assumptions. The key assumption here is that our simulation is an accurate model of reality. Recall that the design of our **Buffon-Laplace** simulation started with a little algebra demonstrating how we could use the ratio of two areas to find the value of $π$. We then translated this idea into code that depended upon a little geometry and on the randomness of `random.random.`

Let’s see what happens if we get any of this wrong. Suppose, for example, we replace the $4$ in the last line of the function `throwNeedles` by a $2$, and again run `estPi(0.01, 100)`. This time it prints


In [20]:
import random

def variance(X):
    """Assumes that X is a list of numbers.
       Returns the standard deviation of X"""
    mean = sum(X)/len(X)
    tot = 0.0
    for x in X:
        tot += (x - mean)**2
    return tot/len(X)
    
def stdDev(X):
    """Assumes that X is a list of numbers.
       Returns the standard deviation of X"""
    return variance(X)**0.5

def throwNeedles(numNeedles):
    inCircle = 0
    for Needles in range(1, numNeedles + 1):
        x = random.random()
        y = random.random()
        if (x*x + y*y)**0.5 <= 1:
            inCircle += 1
    #Counting needles in one quadrant only, so multiply by 4
    #  replace the 4 in the last line of the function throwNeedles by the 2
    return 2*(inCircle/numNeedles)

def getEst(numNeedles, numTrials):
    estimates = []
    for t in range(numTrials):
        piGuess = throwNeedles(numNeedles)
        estimates.append(piGuess)
    sDev = stdDev(estimates)
    curEst = sum(estimates)/len(estimates)
    print('Est. =', str(round(curEst, 5)) + ',',
          'Std. dev. =', str(round(sDev, 5)) + ',',
          'Needles =', numNeedles)
    return (curEst, sDev)

def estPi(precision, numTrials):
    numNeedles = 1000
    sDev = precision
    while sDev > precision/1.96:
        curEst, sDev = getEst(numNeedles, numTrials)
        numNeedles *= 2
    return curEst

In [19]:
precision=0.01
numTrials=100
estPi(precision,numTrials)

Est. = 1.57694, Std. dev. = 0.02357, Needles = 1000
Est. = 1.56774, Std. dev. = 0.01792, Needles = 2000
Est. = 1.5717, Std. dev. = 0.01267, Needles = 4000
Est. = 1.57176, Std. dev. = 0.0096, Needles = 8000
Est. = 1.57085, Std. dev. = 0.00702, Needles = 16000
Est. = 1.57087, Std. dev. = 0.00443, Needles = 32000


1.5708725

The standard deviation for a mere `32,000` needles suggests that we should have a fair amount of confidence in the estimate. But what does that really mean? It means that we can be reasonably confident that if we were to draw more samples from the same distribution, we would get a similar value. It says nothing about whether or not this value is close to the actual value of $\pi$. If you are going to remember only one thing about statistics, remember this: a statistically valid conclusion should not be confused with a correct conclusion!

Before believing the results of a simulation, we need to have confidence both that our conceptual model is correct and that we have correctly implemented that model.Whenever possible, one should attempt to validate results against reality. In this case,one could use some other means to compute an approximation to the area of a circle (e.g., physical measurement) and check that the computed value of $π$ is at least in the
right neighborhood.

## 16.5 Some Closing Remarks About Simulation Models

For most of the history of science, theorists used mathematical techniques to construct purely analytical models that could be used to predict the behavior of a system from a set of parameters and initial conditions.

As the 20th century progressed, the limitations of this approach became increasingly clear. Reasons for this include:

* An increased interest in the social sciences, e.g., economics, led to a desire to construct good models of systems that were not mathematically tractable.

* As the systems to be modeled grew increasingly complex, it seemed easier to successively refine a series of simulation models than to construct accurate analytic models.

* It is often easier to extract useful intermediate results from a simulation than from an analytical model, e.g., to play “what if” games.

* The availability of computers made it feasible to run large-scale simulations. Until the advent of the modern computer in the middle of the 20th century the utility of simulation was limited by the time required to perform calculations by hand. 

Simulation models are **descriptive**, not **prescriptive**. They tell how a system works under given conditions; not how to arrange the conditions to make the system work best. A simulation does not optimize, it merely describes. That is not to say that simulation cannot be used as part of an optimization process. For example, simulation is often used as part of a search process in finding an optimal set of parameter settings.

Simulation models can be classified along three dimensions

* Deterministic versus stochastic,

* Static versus dynamic, and

* Discrete versus continuous.